In [4]:
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [2]:
df2 = pd.read_csv('dataset_192425.csv')

NameError: name 'pd' is not defined

In [4]:
def create_dataframe_sankey(data, value_column, *columns, **filtros):
    for col in columns:
        if col not in data.columns:
            raise ValueError
    
    groupbys = []
    for idx, col in enumerate(columns[:-1]):
        i = data.groupby([columns[idx], columns[idx + 1]])[value_column].sum().reset_index()
        i.columns = ['source', 'target', 'value']
        groupbys.append(i)

    conc = pd.concat(groupbys, ignore_index=True)

    for key, values in filtros.items():
        for value in values:
            conc = conc[conc[key] != value]

    info = enumerate(list(set(conc['source'].unique().tolist() + conc['target'].unique().tolist())))

    dic_info = dict(info)

    rev_info = {}
    for key, value in dic_info.items():
        rev_info[value] = key

    conc['source'] = conc['source'].map(rev_info)
    conc['target'] = conc['target'].map(rev_info)



    return rev_info, conc

In [9]:
df2.columns

Index(['Cuenta', 'Subcuenta', 'Objeto/proyecto', 'Subproyecto',
       'Apropiación en precios corrientes', 'Código de entidad',
       'Tipo de gasto', 'Entidad', 'Código de sector', 'Sector', 'Año',
       'Apropiación en precios constantes (2025)'],
      dtype='object')

In [10]:
lista = ['Entidad', 'Tipo de gasto', 'Cuenta', 'Subcuenta']
create_dataframe_sankey(df2, 
                        'Apropiación en precios constantes (2025)',
                        *lista

                        )

({'Corporación autónoma regional del cauca (crc)': 0,
  'Fortalecimiento de la gestión y dirección del sector fiscalía': 1,
  'Fortalecimiento de la gestión y dirección del sector educación': 2,
  'Unidad administrativa especial del servicio público de empleo': 3,
  'Fortalecimiento de la gestión y dirección del sector ciencia y tecnología': 4,
  'Fortalecimiento de la gestión y dirección del sector transporte': 5,
  'Instituto nacional de formación técnica profesional de san juan del cesar': 6,
  'Promoción de los métodos de resolución de conflictos': 7,
  'Fortalecimiento de la gestión y dirección del sector comunicaciones': 8,
  'Escuela superior de administración pública (esap)': 9,
  'A productores de mercado que distribuyen directamente a los hogares': 10,
  'Fortalecimiento de la infraestructura física de las entidades del estado del nivel nacional desde el sector presidencia': 11,
  'Fortalecimiento de la gestión y dirección del sector empleo público': 12,
  'Instituto nacional

In [5]:
df2 = pd.read_csv('data192425.csv')

In [6]:
df2.columns

Index(['Cuenta', 'Subcuenta', 'Objeto', 'Ordinal', 'TOTAL',
       'Código de entidad', 'Tipo de gasto', 'Entidad', 'Código de sector',
       'Sector', 'Unidad', 'Código de unidad', 'Cuenta_n', 'Subcuenta_n',
       'Objeto_n', 'Ordinal_n', 'Año', 'Indice_base_2025', 'TOTAL_const'],
      dtype='object')

In [7]:
df2.replace({'Subcuenta': {'Prestaciones sociales': 'Prestaciones para cubrir riesgos sociales'},
             'Cuenta':{'Transferencia de capital': 'Transferencias de capital'}}, inplace=True)

In [8]:
df2.to_csv('data192425.csv', index=False)

In [1]:
import pandas as pd
data = pd.read_csv('data192425_hom.csv')

In [104]:
data = pd.read_csv('data192425_hom.csv')

def rename_ord(row):
    entidad = row['Entidad']
    cuenta = row['Cuenta']
    subcuenta = row['Subcuenta']
    objeto = row['Objeto']
    ordinal = row['Ordinal']
    l_objetos = ['PRESTACIONES SOCIALES RELACIONADAS CON EL EMPLEO', 
                 'PRESTACIONES SOCIALES ASUMIDAS POR EL GOBIERNO',
                 'PRESTACIONES DE ASISTENCIA SOCIAL']
    l_ordinal = ['ASEGURAMIENTO, RECLAMACIONES Y SERVICIOS INTEGRALES EN SALUD, (LEY 100 DE 1993 y DECRETO 780 DE 2016)',
                 'APOYO A PROGRAMAS DE DESARROLLO DE LA SALUD LEY 100 DE 1993']
    if entidad == 'Ministerio de hacienda y crédito público' and ordinal == 'OTRAS TRANSFERENCIAS - DISTRIBUCIÓN PREVIO CONCEPTO DGPPN':
        return 'FEPC'
    elif objeto in l_objetos:
        return "PRESTACIONES SOCIALES"
    elif objeto == 'SISTEMA GENERAL DE PARTICIPACIONES':
        return 'SISTEMA GENERAL DE PARTICIPACIONES'
    elif ordinal in l_ordinal:
        return 'Salud'
    else:
        return 'Otros (objeto)'

def rename_cuenta(row):
    cuenta = row['Cuenta']

    if cuenta == 'TRANSFERENCIAS CORRIENTES':
        return 'TRANSFERENCIAS CORRIENTES'
    elif cuenta == 'GASTOS DE PERSONAL':
        return 'GASTOS DE PERSONAL'
    else:
        return 'Otros (cuenta)' 
data['Cuenta_alt'] = data.apply(rename_cuenta, axis=1)
data['Ord_alt'] = data.apply(rename_ord, axis=1)

piv = (data#.query("Objeto == 'SISTEMA GENERAL DE PARTICIPACIONES'").
 .pivot_table(index=['Cuenta_alt', 'Ord_alt'],
                  columns='Año',
                  values='TOTAL_const',
                  aggfunc='sum')
                  .assign(diff_19_24=lambda x: x[2024] - x[2019],
                          diff_19_25=lambda x: x[2025] - x[2019],
                          diff_24_25=lambda x: x[2025] - x[2024])
                    .sort_values(by='diff_19_24', ascending=False).reset_index())

piv = piv[['Cuenta_alt', 'Ord_alt', 'diff_19_24']]
piv.to_excel('datos_william.xlsx', index=False)
cop = piv.copy()

labels = piv['Cuenta_alt'].unique().tolist() + piv['Ord_alt'].unique().tolist()
dic_labels = dict(enumerate(labels))
rev_labels = {}

for key, value in dic_labels.items():
    rev_labels[value] = key


piv['source'] = piv['Cuenta_alt'].map(rev_labels)
piv['target'] = piv['Ord_alt'].map(rev_labels)
piv['values'] = (piv['diff_19_24'] / 1_000_000_000).round(2)

piv = piv.drop(columns=['Cuenta_alt', 'Ord_alt', 'diff_19_24'])

fig = go.Figure(data=[go.Sankey(
    node = dict(
      pad = 15,
      thickness = 20,
      line = dict(color = "black", width = 0.5),
      label = list(rev_labels.keys()),
      color = ["#2F399B"] * 3 +  ["#F7B261"] * 7
    ),
    link = dict(
      source = piv['source'], # indices correspond to labels, eg A1, A2, A1, B1, ...
      target = piv['target'],
      value = piv['values'],
      color = "#D9D9ED"
  ))])

fig.update_layout(title_text="Flujo de gasto - diff (2019 - 2024)", font_size=10)
fig.show()

In [108]:
pie_chart = cop.groupby('Cuenta_alt')['diff_19_24'].sum().reset_index()

,Cuenta_alt,diff_19_24
0,GASTOS DE PERSONAL,1.270304e+13
1,Otros (cuenta),3.609186e+12
2,TRANSFERENCIAS CORRIENTES,7.648788e+13


In [111]:
import plotly.express as px
pie_chart = cop.groupby('Cuenta_alt')['diff_19_24'].sum().reset_index()
fig = px.pie(pie_chart, values='diff_19_24', names='Cuenta_alt', color_discrete_sequence=["#2F399B", "#F7B261", "#0FB7B3"])
fig.show()

In [37]:
for i in data[data['Ordinal'].str.contains('FONDO NACIONAL').fillna(False)]['Ordinal'].unique():
    print(i)


ATENCIÓN DE DESASTRES Y EMERGENCIAS EN EL TERRITORIO NACIONAL -FONDO NACIONAL DE GESTIÓN DEL RIESGO DE DESASTRES
FONDO NACIONAL DE PENSIONES DE LAS ENTIDADES TERRITORIALES LEY 549 DE 1999 (DE PENSIONES)
FONDO NACIONAL DE EMERGENCIAS
FONDO NACIONAL DE PRESTACIONES SOCIALES DEL MAGISTERIO (DE PENSIONES)
FONDO NACIONAL DE PRESTACIONES SOCIALES DEL MAGISTERIO (NO DE PENSIONES)
FONDO DE PENSIONES PÚBLICAS DEL NIVEL NACIONAL - PENSIONES FONDO NACIONAL DE CAMINOS VECINALES (DE PENSIONES)
FONDO NACIONAL DE SEGURIDAD Y CONVIVENCIA CIUDADANA -FONSECON
FONDO NACIONAL PARA LA LUCHA CONTRA LA TRATA DE PERSONAS. LEY 985 DE 2005 Y DECRETO 4319 DE 2006
FONDO NACIONAL DEL PASIVO PENSIONAL Y PRESTACIONAL DE LA ELECTRIFICADORA DEL CARIBE S.A. E.S.P. – FONECA (DE PENSIONES)
FONDO NACIONAL DE EMERGENCIAS SANITARIAS Y FITOSANITARIAS DEL INSTITUTO COLOMBIANO AGROPECUARIO – ICA. 
RECURSOS PARA TRANSFERIR AL FONDO NACIONAL DE PRESTACIONES SOCIALES DEL MAGISTERIO, PREVIA REVISIÓN FALTANTE DE CESANTÍAS


In [38]:
data[data['Ordinal'] == 'FONDO NACIONAL DE PENSIONES DE LAS ENTIDADES TERRITORIALES LEY 549 DE 1999 (DE PENSIONES)']

,Cuenta,Subcuenta,Objeto,Ordinal,TOTAL,Código de entidad,Tipo de gasto,Entidad,Código de sector,Sector,Unidad,Código de unidad,Cuenta_n,Subcuenta_n,Objeto_n,Ordinal_n,Año,Indice_base_2025,TOTAL_const
340,TRANSFERENCIAS CORRIENTES,PRESTACIONES PARA CUBRIR RIESGOS SOCIALES,PRESTACIONES SOCIALES ASUMIDAS POR EL GOBIERNO,FONDO NACIONAL DE PENSIONES DE LAS ENTIDADES T...,1.576380e+11,1301,Funcionamiento,Ministerio de hacienda y crédito público,13,Hacienda,NaN,NaN,3,4.0,3.0,1.0,2019,68.501033,2.301250e+11
2486,TRANSFERENCIAS CORRIENTES,PRESTACIONES PARA CUBRIR RIESGOS SOCIALES,PRESTACIONES SOCIALES ASUMIDAS POR EL GOBIERNO,FONDO NACIONAL DE PENSIONES DE LAS ENTIDADES T...,1.234510e+11,1301,Funcionamiento,Ministerio de hacienda y crédito público,13,Hacienda,NaN,NaN,3,4.0,3.0,1.0,2024,94.339623,1.308581e+11
4629,TRANSFERENCIAS CORRIENTES,PRESTACIONES PARA CUBRIR RIESGOS SOCIALES,PRESTACIONES SOCIALES ASUMIDAS POR EL GOBIERNO,FONDO NACIONAL DE PENSIONES DE LAS ENTIDADES T...,2.172738e+11,1301,Funcionamiento,Ministerio de hacienda y crédito público,13,Hacienda,MINISTERIO DE HACIENDA Y CRÉDITO PÚBLICO - GES...,130101.0,3,4.0,3.0,1.0,2025,100.000000,2.172738e+11


In [25]:
data['Indice_base_2025'].unique()

array([ 68.501033  ,  94.33962264, 100.        ])

In [26]:
6.026999e+13 * (68.501033 / 100)

41285565738996.7